In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
df = pd.read_csv('df_M30.csv', delimiter=';')

In [ ]:
new_df = df[df['oldID'].str.contains('PM1.*1$')]

In [ ]:
new_df = new_df.loc[new_df['oldID'].str.len() == 7]

In [ ]:
new_df['KM'] = new_df["oldID"].str[3:5].astype(str) + "." + new_df["oldID"].str[5:6].astype(str)

In [ ]:
new_df = new_df[new_df['fecha'].str.contains('2015-08-04')]

In [ ]:
new_df[['KM']] = new_df.loc[:,['KM']].astype(np.float64)

In [ ]:
new_df.loc[:, 'densidad'] = new_df.loc[:, "intensidad"] / new_df.loc[:, "vmed"]

In [ ]:
new_df.dropna(subset = ['densidad'])

In [ ]:
new_df['Hora'] = new_df["fecha"].str[11:13].astype(str) + "." + new_df["fecha"].str[14:16].astype(str)
new_df[['Hora']] = new_df.loc[:,['Hora']].astype(np.float64)

In [ ]:
new_df.sort_values('Hora').reset_index(drop=True)

In [ ]:
new_df.plot.scatter(x="densidad", y="Hora")

In [ ]:
display(new_df)

In [ ]:
new_df.dtypes

In [ ]:
new_df.to_csv('df_M30_KM_dia.csv', sep=';')

# Now work with onde day data_base

In [2]:
df = pd.read_csv('df_M30_KM_dia.csv', delimiter=';')

In [4]:
display(df)

,Unnamed: 0,ID,oldID,fecha,intensidad,ocupacion,vmed,periodo_integracion,KM,densidad,Hora
0,2288210,6672,PM10941,2015-08-04 04:45,280.0,0.0,91.0,15,9.4,3.076923,4.45
1,2288214,6676,PM10981,2015-08-04 04:45,420.0,1.0,87.0,15,9.8,4.827586,4.45
2,2288215,6693,PM12241,2015-08-04 04:45,108.0,0.0,51.0,15,22.4,2.117647,4.45
3,2288216,6694,PM12331,2015-08-04 04:45,117.0,4.0,62.0,15,23.3,1.887097,4.45
4,2288218,6695,PM12391,2015-08-04 04:45,144.0,1.0,64.0,15,23.9,2.250000,4.45
5,2288219,6696,PM12471,2015-08-04 04:45,119.0,0.0,84.0,15,24.7,1.416667,4.45
6,2288223,6699,PM12641,2015-08-04 04:45,68.0,0.0,63.0,15,26.4,1.079365,4.45
7,2288224,6700,PM12721,2015-08-04 04:45,76.0,0.0,65.0,15,27.2,1.169231,4.45
8,2288225,3801,PM12781,2015-08-04 04:45,79.0,0.0,80.0,15,27.8,0.987500,4.45
9,2288226,6914,PM12851,2015-08-04 04:45,80.0,4.0,55.0,15,28.5,1.454545,4.45


In [5]:
df.describe()

,Unnamed: 0,ID,intensidad,ocupacion,vmed,periodo_integracion,KM,densidad,Hora
count,3.973000e+03,3973.000000,3973.000000,3973.000000,3973.000000,3973.000000,3973.000000,3828.000000,3973.000000
mean,2.476338e+06,6351.379814,1712.572363,5.768437,75.808457,14.990939,17.161616,22.200429,11.667103
std,2.333664e+05,968.704486,1461.973819,5.528261,20.847811,0.310797,11.044473,18.303454,6.924018
min,2.288210e+06,3510.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.659574,0.000000
25%,2.328270e+06,6648.000000,425.000000,2.000000,68.000000,15.000000,5.000000,7.041270,5.450000
50%,2.376677e+06,6687.000000,1336.000000,5.000000,82.000000,15.000000,20.600000,18.985307,11.450000
75%,2.428775e+06,6699.000000,2653.000000,8.000000,89.000000,15.000000,27.800000,32.397969,17.450000
max,2.940426e+06,7130.000000,7340.000000,85.000000,105.000000,15.000000,32.200000,161.100000,23.450000


In [27]:
# df[df['periodo_integracion'] == 1] # 1 muestra (958)
# df[df['periodo_integracion'] == 3] # 1 muestra (3508)
# f[df['periodo_integracion'] == 9] # 1 muestra (3239)
# df[df['periodo_integracion'] == 13] # 2 muestras (1560 y 2222)


,Unnamed: 0,ID,oldID,fecha,intensidad,ocupacion,vmed,periodo_integracion,KM,densidad,Hora
3239,2919241,6642,PM10091,2015-08-04 00:30,1440.0,20.0,36.0,9,0.9,40.0,0.3


In [ ]:
# Add a new row at index position 2 with values provided in list
df.iloc[2] = [0, 6672, , 'India']